# GTEx v7 - Download and Basic library preparation

Downloads the files.  Shows data and some plot

In [2]:
# Graphing library imports

import os, sys, glob, logging
import inspect, urllib

import pandas as pd
import numpy as np
import scipy
from pprint import pprint

import time, datetime

import matplotlib.pyplot as plt
import seaborn as sns

# __file__ does not work in a jupyter notebook
# make something equivalent - actually only for this particular cell
import inspect
# __file__ does not work in a jupyter notebook
# make something equivalent - actually only for this particular cell
__file__ = os.path.abspath(inspect.getfile(lambda: None))
import pathlib
script_dir = pathlib.Path().resolve()
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
# Data files - Find the source GTEX data 
# Download to a cache
SIMPLE_FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
DATEFMT = '%Y-%m-%d %H:%M:%S'
logging.basicConfig(level=logging.INFO, format=SIMPLE_FORMAT, datefmt=DATEFMT)
logging.info("logging")

# GTEx data from https://gtexportal.org/home/datasets
# Gene transcripts per million data
cache_dir_alt = os.path.join(pathlib.Path().resolve(), 'gtex', 'data_cache')
cache_dir = '/gsc/resources/expression/GTEx/v7'
if not os.path.exists(cache_dir) and os.path.exists(cache_dir_alt):
    cache_dir = cache_dir_alt
if not os.path.exists(cache_dir):
    logging.info('Creating {}'.format(cache_dir))
    os.makedirs(cache_dir, mode=0o777)

hgnc_fns = glob.glob(os.path.join(cache_dir, 'hgnc*'))
if not hgnc_fns:
    hgnc_fn = os.path.join(cache_dir, 'hgnc_{}.tsv'.format(
        datetime.date.today().strftime('%Y%m%d')))
    logging.info('Downloading HGNC annotations to: %s', hgnc_fn)
    urllib.request.urlretrieve(
        "ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt", hgnc_fn)
hgnc_fns = glob.glob(os.path.join(cache_dir, 'hgnc*'))
hgnc = pd.read_csv(hgnc_fns[0], sep='\t')

# Detailed data file:
# This file is quite large ~ 1GB
gtex_tpm_link = 'https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz'

# Try Using unzipped version
gtex_tpm_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct')
if not os.path.exists(gtex_tpm_fn):
    gtex_tpm_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz')
if not os.path.exists(gtex_tpm_fn):
    print("downloading {}".format(gtex_tpm_link))
    urllib.request.urlretrieve(gtex_tpm_link, gtex_tpm_fn)

# Just looking at median values by tissue (SMTSD) greatly reduces file size
gtex_tpm_med_link = 'https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct.gz'
# Try using unzipped version
gtex_tpm_med_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct')
if not os.path.exists(gtex_tpm_med_fn):
    gtex_tpm_med_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct.gz')
if not os.path.exists(gtex_tpm_med_fn):
    print("downloading {}".format(gtex_tpm_med_link))
    urllib.request.urlretrieve(gtex_tpm_med_link, gtex_tpm_med_fn)

# Sample annotation data

# Subject Phenotype data
# Age, sex and Hardy Scale death circumstances
gtex_pheno_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SubjectPhenotypesDS.txt'
gtex_pheno_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SubjectPhenotypesDS.txt')

# Main sample data of interest is:
# 'SMTS': 'Tissue Type, area from which the tissue sample was taken.  This is a parent value to SMTSD.'
# 'SMTSD': 'SMTS Detailed'
gtex_attr_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt'
gtex_attr_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SampleAttributesDS.txt')
gtex_attr_desc_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx'
gtex_attr_desc_fn = os.path.join(cache_dir, 'GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx')
if not os.path.exists(gtex_attr_fn):
    logging.info('downloading {}'.format(gtex_attr_link))
    urllib.request.urlretrieve(gtex_attr_link, gtex_attr_fn)
if not os.path.exists(gtex_attr_desc_fn):
    logging.info('downloading {}'.format(gtex_attr_desc_link))
    urllib.request.urlretrieve(gtex_attr_desc_link, gtex_attr_desc_fn)

# Load tissue specific detatils
gtex_attr = pd.read_csv(gtex_attr_fn, sep='\t')
tissue_types = sorted(gtex_attr['SMTS'].unique())
tissue_dict = dict()
for tissue in tissue_types:
    tissue_dict[tissue] = dict()
    tissue_dict[tissue]['samples'] = gtex_attr[gtex_attr['SMTS'] == tissue]['SAMPID'].to_list()
    tissue_dict[tissue]['subtypes'] = dict()
    for subtype in gtex_attr[gtex_attr['SMTS'] == tissue]['SMTSD'].unique():
        # Get all samples by subtype
        tissue_dict[tissue]['subtypes'][subtype] = gtex_attr[gtex_attr['SMTSD'] == subtype]['SAMPID'].to_list()
    

    sorted(gtex_attr[gtex_attr['SMTS'] == tissue]['SMTSD'].unique())
tissue_subtypes = sorted(gtex_attr['SMTSD'].unique())

# log number of tissue samples
logging.info('%s separate tissue types - total samples %s', len(tissue_dict), len(gtex_attr['SAMPID']))
for tissue in sorted(tissue_dict.keys()):
    logging.info('%s (n=%s)', tissue, len(tissue_dict[tissue]['samples']))
    if len(tissue_dict[tissue]['subtypes']) > 1:
        for subtype in sorted(tissue_dict[tissue]['subtypes'].keys()):
            logging.info('\t%s (n=%s)', subtype, len(tissue_dict[tissue]['subtypes'][subtype]))
# logging.info(pprint.pformat(tissue_dict))

# Other annotation data

# Mutually exclusive 
# Which study was this from?
mut_info_fn = os.path.join(cache_dir, 'enrichments-analysis-result.txt')
#mut_info = pd.read_csv(mut_info_fn, sep='\t')

# Human housekeeping gene - stable expression

## https://www.cell.com/trends/genetics/pdf/S0168-9525(13)00089-9.pdf - Human housekeeping genes, revisited

# https://m.tau.ac.il/~elieis/HKG/HK_genes.txt
# E. Eisenberg and E.Y. Levanon, Trends in Genetics, 29 (2013)

2019-08-21 23:14:06 - root - INFO - logging
/home/zeno/miniconda3/envs/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (32,34,38,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
2019-08-21 23:14:08 - root - INFO - 31 separate tissue types - total samples 15598
2019-08-21 23:14:08 - root - INFO - Adipose Tissue (n=886)
2019-08-21 23:14:08 - root - INFO - 	Adipose - Subcutaneous (n=523)
2019-08-21 23:14:08 - root - INFO - 	Adipose - Visceral (Omentum) (n=363)
2019-08-21 23:14:08 - root - INFO - Adrenal Gland (n=205)
2019-08-21 23:14:08 - root - INFO - Bladder (n=11)
2019-08-21 23:14:08 - root - INFO - Blood (n=2561)
2019-08-21 23:14:08 - root - INFO - 	Cells - EBV-transformed lymphocytes (n=149)
2019-08-21 23:14:09 - root - INFO - 	Whole Blood (n=2412)
2019-08-21 23:14:09 - root - INFO - Blood Vessel (n=1041)
2019-08-21 23:14:09 - root - INFO - 

In [21]:
# Some table loading helper functions

# The full set of all all GTEx tpms is very large and can be hard to work with
# Some functions only load some of the data at a time.

# Genes of interest - some test genes 
TEST_GENES = [
    'TP53',
    'ERBB2',  # Herceptin target
    'EGFR',
    'AKT1',
    'KRAS',
    'PTEN',
    'APOE',
]

TUMOUR_MUT_GENES = [  # www.tumourportal.org - highly mutated
    'TP53',
    'PIK3CA',
    'PTEN',
    'KRAS',
    'APC',
    'MLL3',  # aka KMT2C
    'KMT2C',
    'FAT1',
    'MLL2',  # aka KMT2D
    'KMT2D',
    'ARID1A',
    'VHL',
    'PBRM1',
    'NF1',
    'EGFR',
    'ATM',
    'PIK3R1',
    'BRAF',
    'CDKN2A',
    'SETD2',
    'CREBBP',
    'FBXW7',
    'SPEN',
    'MTOR',
    'RB1',
    'SMARCA4',
    'NOTCH1',
]

TEST_GENES = sorted(set(TEST_GENES + TUMOUR_MUT_GENES))

ts = []
onco = []
with open(os.path.join(script_dir, 'tables', 'TS_genes_in_kb.txt')) as ifile:
    for line in ifile:
        if line.strip():
            ts.append(line.strip())
with open(os.path.join(script_dir, 'tables', 'oncogenes_in_kb.txt')) as ifile:
    for line in ifile:
        if line.strip():
            onco.append(line.strip())

ts_onco = sorted(set(ts + onco + TEST_GENES))

# gtex_tpm file has genes as rows.
# First two columns are gene information
# Over 10000 columns of samples

#                Name Description  GTEX-1117F-0226-SM-5GZZ7  GTEX-111CU-1826-SM-5GZYN ...
#0  ENSG00000223972.4     DDX11L1                   0.10820                   0.11580
#1  ENSG00000227232.4      WASH7P                  21.40000                  11.03000


GTPM_SKIP = 2  #  skip two comment rows at the beginning of the file

# Just getting a few rows of this giant file
logging.info('Loading Row partial table ...')
gtpm_row_table = pd.read_csv(gtex_tpm_fn, skiprows=2, sep='\t', usecols=range(4))
logging.info('Loading column partial table ...')
gtpm_column_table = pd.read_csv(gtex_tpm_fn, skiprows=lambda x: x not in [2, 3], sep='\t')

def gtpm_row_by_genename(gene_name):
    sub_table = gtpm_row_table[gtpm_row_table['Description'] == gene_name]
    if sub_table.empty:
        logging.error("Missing '%s row in GTEX TPM table'", gene_name)
        return None
    return [GTPM_SKIP + 1 + i for i in sub_table.index][0]

def gtpm_keep_rows(gene_list=TEST_GENES):
    row_list = [gtpm_row_by_genename(gene) for gene in gene_list]
    return [GTPM_SKIP] + [gene for gene in row_list if gene]


def gtex_tpm_partial_table_load(gene_list, sample_list=None):
    start_time = time.time()
    if gene_list:
        keep_rows = gtpm_keep_rows(gene_list)
        skiprows_func = lambda x: x not in keep_rows
        logging.info('loading %s GTEX TPM rows', len(keep_rows))
    else:
        logging.info('loading all rows')
        skiprows_func = lambda x: x in [0, 1]
    
    if sample_list:
        logging.info('Limiting load to %s potential samples', len(sample_list))
        # Not all samples are in tables
        sample_list = [s for s in sample_list if s in gtpm_column_table.columns]
        logging.info('Limiting load to %s samples', len(sample_list))
        columns = ['Name', 'Description'] + sample_list
    else:
        sample_list = None
    
    gtpm_partial = pd.read_csv(
        gtex_tpm_fn,
        skiprows=skiprows_func,  # Only selected gene rows
        sep='\t',
        header=0,
        usecols=sample_list,  # Number of samples limit
    )
    end_time = time.time()
    logging.info("Gtex TPM partial table took {} minutes".format((end_time - start_time)/60))
    return gtpm_partial


2019-08-21 23:38:39 - root - INFO - Loading Row partial table ...
2019-08-21 23:39:06 - root - INFO - Loading column partial table ...


In [5]:
# Small table of median values per tissue only
gtpm_med = pd.read_csv(gtex_tpm_med_fn, sep='\t', skiprows=2, low_memory=False)
# gtex log2
g2_fn = os.path.join(cache_dir, 'preprocess', 'GTEX_log2.tsv')
logging.info('Loading g2 row partial table ...')
g2_row_table = pd.read_csv(g2_fn, sep='\t', usecols=range(4))
logging.info('Loading g2 column partial table ...')
g2_col_table = pd.read_csv(g2_fn, skiprows=lambda x: x not in [0, 1, 2], sep='\t')
logging.info('done')

2019-08-21 23:16:17 - root - INFO - Loading g2 row partial table ...
2019-08-21 23:17:09 - root - INFO - Loading g2 column partial table ...
2019-08-21 23:18:02 - root - INFO - done


In [15]:
g2_samples = list(g2_col_table.columns[3:])
print(g2_row_table.head())
#print(g2_col_table.head())
print(len(g2_row_table))

       Symbol            ensid  ens_ver  GTEX-1117F-0226-SM-5GZZ7
0     DDX11L1  ENSG00000223972        4                 -3.208228
1      WASH7P  ENSG00000227232        4                  4.419539
2  MIR1302-11  ENSG00000243485        2                 -2.642054
3     FAM138A  ENSG00000237613        2                 -4.309002
4      OR4G4P  ENSG00000268020        2                       NaN
30214


In [20]:
g2_samples = list(g2_col_table.columns[3:])
td = tissue_dict.copy()
for tissue in td.keys():
    td[tissue]['samples'] = [s for s in td[tissue]['samples'] if s in g2_samples]
    for subtype in td[tissue]['subtypes'].keys():
        td[tissue]['subtypes'][subtype] = [s for s in td[tissue]['subtypes'][subtype] if s in g2_samples]
for tissue in sorted(td.keys()):
    logging.info('%s (n=%s)', tissue, len(td[tissue]['samples']))
    if len(td[tissue]['subtypes']) > 1:
        for subtype in sorted(td[tissue]['subtypes'].keys()):
            logging.info('\t%s (n=%s)', subtype, len(td[tissue]['subtypes'][subtype]))

2019-08-21 23:36:40 - root - INFO - Adipose Tissue (n=797)
2019-08-21 23:36:40 - root - INFO - 	Adipose - Subcutaneous (n=442)
2019-08-21 23:36:40 - root - INFO - 	Adipose - Visceral (Omentum) (n=355)
2019-08-21 23:36:40 - root - INFO - Adrenal Gland (n=190)
2019-08-21 23:36:40 - root - INFO - Bladder (n=11)
2019-08-21 23:36:40 - root - INFO - Blood (n=537)
2019-08-21 23:36:40 - root - INFO - 	Cells - EBV-transformed lymphocytes (n=130)
2019-08-21 23:36:40 - root - INFO - 	Whole Blood (n=407)
2019-08-21 23:36:40 - root - INFO - Blood Vessel (n=913)
2019-08-21 23:36:40 - root - INFO - 	Artery - Aorta (n=299)
2019-08-21 23:36:40 - root - INFO - 	Artery - Coronary (n=173)
2019-08-21 23:36:40 - root - INFO - 	Artery - Tibial (n=441)
2019-08-21 23:36:40 - root - INFO - Bone Marrow (n=0)
2019-08-21 23:36:40 - root - INFO - Brain (n=1671)
2019-08-21 23:36:40 - root - INFO - 	Brain - Amygdala (n=100)
2019-08-21 23:36:40 - root - INFO - 	Brain - Anterior cingulate cortex (BA24) (n=121)
2019-08-

In [ ]:

# Apply some known values
gm['mean'] = gm.apply(lambda row: row.mean(), axis=1)
gm['median'] = gm.apply(lambda row: row.median(), axis=1)
gm['std'] = gm.apply(lambda row: row.std(), axis=1)
#gm['mode'] = gm.apply(lambda row: row.mode(), axis=1)
gm['CoV'] = gm.apply(lambda row: pandas.np.inf if row['mean'] == 0 else row['std']/row['mean'], axis=1)
gm['sem'] = gm.apply(lambda row: row.sem(), axis=1)
gm.head()

# high expression # low deviation among tissues
gm_bench = gm[(gm['median'] >= 1)  # high expression
             & (gm['median'] - 3.5 * gm['std'] > 0)
            ]

#cross_tissue_genes = sorted(gm_bench['Description'].tolist())

print("Of {} genes, the top {} in stable cross-tissue expression are:".format(len(gm), len(gm_bench)))
#for gene in cross_tissue_genes:
#    print('\t{}'.format(gene))
gm_bench.sort_values(by='CoV', inplace=True)
gm_bench


In [ ]:
gm.sort_values(by='sem').head(30)

In [ ]:
gm.loc[TEST_GENES]

In [ ]:
# Transpose to get Gene Columns
gt = gtpm[gtpm.columns[1:]]
gt.set_index('Description', inplace=True)
org_cols = gt.columns

#gtt = gt.transpose()
#gtt.head()
gt.head()

In [ ]:
print(gt.loc['TP53'].describe())
print(gt.loc['ERBB2'].describe())

In [ ]:
#gt.loc['TP53'].value_counts(bins=100, sort=False)
#gt.loc['PTEN'].skew()
#gt.loc['PTEN'].kurtosis()
gt.loc['PTEN'].quantile(0.9)
gt.T[TEST_GENES[:-1]].iplot(kind='box',)
gt.T[['TP53', 'PTEN']].plot(kind='box',)

In [ ]:
#gene = 'TP53'
gene = 'PTEN'
title = '{}: Samples {}'.format(gene, len(gt.loc[gene]))
#gtt[gene].iplot(kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)
# gt.loc[gene].iplot(kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)
# gt.loc[gene].plot(kind='hist', title=title, bins=100)

sns.set()
fig = plt.figure()
plt.ylabel('counts')
sns.distplot(gt.loc[gene], kde=False, rug=False)
out_filename = os.path.join(script_dir, 'img', gene + '_raw.png')
plt.title(gene + ' skew: {} kurtosis: {}'.format(round(gt.loc[gene].skew(), 2),
                                              round(gt.loc[gene].kurtosis(), 2)))
fig.savefig(out_filename)
fig = plt.figure()

ptenlog2 = gt.loc[gene].apply(numpy.log2)
plt.title('log2({}) skew: {} kurtosis: {}'.format(gene,
                                                  round(ptenlog2.skew(), 2),
                                                  round(ptenlog2.kurtosis(), 2)))
sns.distplot(ptenlog2, kde=False, rug=False)
plt.ylabel('counts')
plt.xlabel(gene + ' log2')
out_filename = os.path.join(script_dir, 'img', gene + '_log2.png')
fig.savefig(out_filename)

In [ ]:
gene = 'TP53'

title = '{}: Samples {}'.format(gene, len(gt.loc[gene]))
#gtt[gene].iplot(kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)
# gt.loc[gene].iplot(kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)
# gt.loc[gene].plot(kind='hist', title=title, bins=100)

sns.set()
fig = plt.figure()
plt.ylabel('counts')
sns.distplot(gt.loc[gene], kde=False, rug=False)
out_filename = os.path.join(script_dir, 'img', gene + '_raw.png')
plt.title(gene + ' skew: {} kurtosis: {}'.format(round(gt.loc[gene].skew(), 2),
                                              round(gt.loc[gene].kurtosis(), 2)))
fig.savefig(out_filename)
fig = plt.figure()

ptenlog2 = gt.loc[gene].apply(numpy.log2)
plt.title('log2({}) skew: {} kurtosis: {}'.format(gene,
                                                  round(ptenlog2.skew(), 2),
                                                  round(ptenlog2.kurtosis(), 2)))
sns.distplot(ptenlog2, kde=False, rug=False)
plt.ylabel('counts')
plt.xlabel(gene + ' log2')
out_filename = os.path.join(script_dir, 'img', gene + '_log2.png')
fig.savefig(out_filename)

In [ ]:
#gt.head()
# sns.distplot(gt.loc[gene], kde=False, rug=False,)
fig = plt.figure()

for tissue in ['Heart', 'Nerve', 
               'Skin',
               'Lung',
               'Pancreas'
              ]:
    samples = tissue_dict[tissue]['samples']
    # print(tissue)
    # print(len(samples))
    gt_samples = [s for s in samples if s in gt.columns]
    dt = gt.loc[gene][gt_samples].dropna().apply(numpy.log2)
    sns.distplot(dt, kde=True, rug=False, label=tissue)
plt.legend()
plt.title('log2(TP53) on some tissues')
out_filename = os.path.join(script_dir, 'img', gene + '_by_tissue.png')
fig.savefig(out_filename)

In [ ]:
print(plotly.__version__)
print(cufflinks.__version__)
#print(ff.create_distplot.__doc__)
gene_list = ['TP53', 'PTEN', 'KRAS', 'ERBB2', 'CDKN2A']
gene_list = ['TP53', 'PTEN', 'KRAS',]
fig = ff.create_distplot([gt.loc[g] for g in gene_list], gene_list)
iplot(fig)

In [ ]:
# by tissue types
gene = 'TP53'
data_lists = []
labels = []
for tissue in tissue_dict.keys():
    samples = tissue_dict[tissue]['samples']
    # print(tissue)
    # print(len(samples))
    gt_samples = [s for s in samples if s in gt.columns]
    if len(gt_samples) > 20:
        dt = gt.loc[gene][gt_samples].dropna()
        if not dt.empty:
            data_lists.append(dt)
            labels.append('{}({}) ~ {}'.format(tissue, len(samples), round(dt.median(), 1)))
    else:
        print('Only {} samples for "{}"'.format(len(gt_samples), tissue))
# add all samples
dt = gt.loc[gene].dropna()
data_lists.append(dt)
labels.append('{} - all ({}) ~ {}'.format(gene, len(dt), round(dt.median(), 1)))

fig_tp53 = ff.create_distplot(data_lists, labels, show_hist=False, show_rug=False, show_curve=True)  # curve_type='normal')
fig_tp53.layout.update(title=gene)
iplot(fig_tp53)

In [ ]:
# Expected distributions are roughly log normal - not normal distributions.
dir(go)
gene='PTEN'
tpm_label = "TPM      {} skew: {} kurtosis: {}".format(gene,
                                                       round(gt.loc[gene].skew(), 2),
                                                       round(gt.loc[gene].kurtosis(), 2))
log2_label = "log2TPM {} skew: {} kurtosis: {}".format(gene,
                                                       round(gt.loc[gene].apply(numpy.log2).skew(), 2),
                                                       round(gt.loc[gene].apply(numpy.log2).kurtosis(), 2))

fig_log2vsNormal = ff.create_distplot([gt.loc[gene].apply(numpy.log2), gt.loc[gene]],
                         [log2_label, tpm_label],
                         bin_size=[0.1, 2],
                         curve_type='normal',)
iplot(fig_log2vsNormal)



# Data structure

Data like TPM is lognormal distributed. 
 * There cannot be a negative number of transcripts.
 * log transforms (like fold-change - log 2) are more normally disturbuted.
 
 More irregular curves, like TP53 show more log-normal distributions when clustered into relevent groups, like tissue types.
 
 Otherwise, something like sklearn.preprocessing.RobustScaler looks good for IQR scaling - robust to outliers.
 
 This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).
 
 
 ## Correlation
 
 Gene expressions are correlated / anti-correlated (mutual information/effect) in interaction pathways.  'Activation' of certiain pathways is of interest for cancer and drug effects measures.
 
 
 PCA with 'whitening' 

In [ ]:
# Normalize to log2RPKM - log normal data
# Data is now effectively in fold change units

# Use a minum value to prevent div by zero.
#safelog2 = lambda x: numpy.log2(x) if x else numpy.log2(sys.float_info.min)
safelog2 = lambda x: numpy.log2(x) if x else numpy.nan
gfc = gt.applymap(safelog2)

print(gfc.info())
print(gfc.head())

In [ ]:
# Try data reduction like PCA and LDA

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler

# Which way are we scaling? Rows or columns?
rs = RobustScaler()
scaler = rs.fit(gfc.T)
gfc_norm = pandas.DataFrame(scaler.transform(gfc.T), index=gfc.columns, columns=gfc.index)
print(gfc_norm.head())
gfc_norm.describe()


In [ ]:
# by tissue types
gene = 'PTEN'
data_lists = []
labels = []
for tissue in tissue_dict.keys():
    samples = tissue_dict[tissue]['samples']
    # print(tissue)
    # print(len(samples))
    gt_samples = [s for s in samples if s in gfc_norm.index]
    if len(gt_samples) > 20:
        dt = gfc_norm[gene][gt_samples].dropna()
        if not dt.empty:
            data_lists.append(dt)
            labels.append('{}({}) ~ {}'.format(tissue, len(samples), round(dt.median(), 1)))
    else:
        print('Only {} samples for "{}"'.format(len(gt_samples), tissue))
# add all samples
dt = gfc_norm[gene].dropna()
data_lists.append(dt)
labels.append('{} - all ({}) ~ {}'.format(gene, len(dt), round(dt.median(), 1)))

fig_log2tissue = ff.create_distplot(data_lists, labels, show_hist=False, show_rug=False, show_curve=True)  # curve_type='normal')
fig_log2tissue.layout.update(title=gene)
iplot(fig_log2tissue)

In [ ]:
# imitate https://stackabuse.com/implementing-lda-in-python-with-scikit-learn/
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

import sklearn.decomposition
print('sklearn.decomposition')
pprint([f for f in dir(sklearn.decomposition) if str(f)[0] != '_'])

import sklearn.discriminant_analysis
print('sklearn.discriminant_analysis')
pprint([f for f in dir(sklearn.discriminant_analysis) if str(f)[0] != '_'])

print('sklearn.ensemble')
import sklearn.ensemble
pprint([f for f in dir(sklearn.ensemble) if str(f)[0] != '_'])

print('sklearn.metrics')
import sklearn.metrics
pprint([f for f in dir(sklearn.metrics) if str(f)[0] != '_'])
pprint(dir(sklearn.metrics))



In [ ]:
# PCA
# sparse PCA is probably better - interpret which genes are more important.

pca3 = PCA(n_components=3)  # Alt opt PCA(0.99) - enough components for 99% of variance
sparse_pca = sklearn.decomposition.SparsePCA(n_components=3, normalize_components=True)
sparse_comp = sparse_pca.fit_transform(gfc_norm)
components3 = pca3.fit_transform(gfc_norm)

print(pandas.DataFrame(components3).head())
pca3.explained_variance_ratio_

In [ ]:
# Find the most important genes in pc1

sparse_comp = pandas.DataFrame(sparse_pca.components_, columns=gfc_norm.columns)
print(sparse_comp)
pc1 = sparse_comp.loc[0]
#pc1_genes = [g.index for g in pc1 if g != 0]
print(pc1)
pc1 = pc1.apply(abs)
#print(pc1.sort_values.__doc__)
pc1.sort_values(ascending=False, inplace=True)
print(pc1)
pc1_gene_list =[gene for gene in pc1.index if pc1[gene] != 0]
print(pc1_gene_list)


In [ ]:
# Some playing around with mutiindex for more useful tables
gtm = pandas.read_csv(gtex_tpm_med_fn, sep='\t', skiprows=2, low_memory=False)
gtm.set_index(['Description', 'gene_id'], inplace=True)
#tissues = [(col.split(' - ')[0], col.replace(col.split(' - ')[0] + ' - ', '')) for col in gtm.columns]
tissues = [col.split(' - ')[0] for col in gtm.columns]
subtypes =  [col.replace(col.split(' - ')[0] + ' - ', '') for col in gtm.columns]

gtm.columns = [tissues, subtypes]
print(gtm.head())

print(gtm.loc['TP53', 'Brain'])

# Makes some reasonable subtables.  With droplevel seems to work ok.

In [ ]:
gtm.loc[:, 'Brain']
#gtm.loc[:, gtm.columns.get_level_values(1) == 'Cortex']

In [ ]:
#gtm['skew_Adipose'] = gtm.loc[:, 'Adipose'].apply(lambda row.skew(), axis=1)
len(gtm.columns[0])



In [ ]:
test = gtex_tpm_partial_table_load(gene_list=['TP53'], sample_list=tissue_dict['Lung']['samples'])
len(test)



In [ ]:
tissue_dict['Brain']['subtypes']['Brain - Hippocampus']


In [ ]:
gg = gt[list(gt.columns[:20])]
gg.head()


In [ ]:
gmt = gtpm_med.set_index(['Description', 'gene_id']).T

In [ ]:
gmd = gmt.describe()

In [ ]:
low_tissue_expression = gmd.T[gmd.T['75%'] <= 0.01]
len(low_tissue_expression)

In [ ]:
# log2 table for measurement
g2 = pandas.read_csv('/gsc/resources/expression/GTEx/v7/preprocess/GTEX_log2.tsv', sep='\t')

In [ ]:
g2.head()